In [1]:
import pandas as pd
df = pd.read_csv("names-by-nationality.csv")
df

,name,sex,nationality
0,Aarav,Male,Indian
1,Aaryan,Male,Indian
2,Abha,Female,Indian
3,Abhay,Male,Indian
4,Abhilash,Male,Indian
...,...,...,...
4690,Worth,Male,American
4691,Wrigley,Neutral,American
4692,X-iomania,Female,American
4693,Xadrian,Male,American


In [3]:
df.head()

,name,sex,nationality
0,Aarav,Male,Indian
1,Aaryan,Male,Indian
2,Abha,Female,Indian
3,Abhay,Male,Indian
4,Abhilash,Male,Indian


In [23]:
df.tail()

,name,sex,nationality
4690,Worth,Male,American
4691,Wrigley,Neutral,American
4692,X-iomania,Female,American
4693,Xadrian,Male,American
4694,Xakery,Male,American


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4695 entries, 0 to 4694
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         4695 non-null   object
 1   sex          4695 non-null   object
 2   nationality  4695 non-null   object
dtypes: object(3)
memory usage: 110.2+ KB


In [25]:
df.shape

(4695, 3)

In [27]:
df.isnull().sum()

name           0
sex            0
nationality    0
dtype: int64

In [29]:
df['name'].isnull().sum()

np.int64(0)

In [31]:
df['sex'].isnull().sum()

np.int64(0)

In [33]:
df['nationality'].isnull().sum()

np.int64(0)

In [37]:
df.duplicated().sum()

np.int64(0)

In [39]:
df['name'].duplicated().sum()

np.int64(0)

In [41]:
#df['sex'].duplicated().sum()

np.int64(4692)

In [43]:
#df['nationality'].duplicated().sum()

np.int64(4688)

In [51]:
df = df.drop_duplicates()

In [54]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['sex_label'] = le.fit_transform(df['sex'])  # Male=1, Female=0, Neutral=2 (example)


In [56]:
df

,name,sex,nationality,sex_label
0,Aarav,Male,Indian,1
1,Aaryan,Male,Indian,1
2,Abha,Female,Indian,0
3,Abhay,Male,Indian,1
4,Abhilash,Male,Indian,1
...,...,...,...,...
4690,Worth,Male,American,1
4691,Wrigley,Neutral,American,2
4692,X-iomania,Female,American,0
4693,Xadrian,Male,American,1


In [58]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize characters (not words)
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(df['name'])

# Convert names to sequences
sequences = tokenizer.texts_to_sequences(df['name'])

# Pad sequences to same length
max_len = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_len, padding='post')

y = df['sex_label'].values


In [59]:
df

,name,sex,nationality,sex_label
0,Aarav,Male,Indian,1
1,Aaryan,Male,Indian,1
2,Abha,Female,Indian,0
3,Abhay,Male,Indian,1
4,Abhilash,Male,Indian,1
...,...,...,...,...
4690,Worth,Male,American,1
4691,Wrigley,Neutral,American,2
4692,X-iomania,Female,American,0
4693,Xadrian,Male,American,1


In [106]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# One-hot encode the labels
y_cat = to_categorical(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# Model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=16, input_length=max_len))
model.add(SimpleRNN(64, return_sequences=False))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_cat.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)



C:\Users\ahsan\anaconda3\envs\envpro\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 18s 32ms/step - accuracy: 0.5506 - loss: 0.9893 - val_accuracy: 0.5687 - val_loss: 0.9500
Epoch 2/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.5735 - loss: 0.9221 - val_accuracy: 0.5729 - val_loss: 0.9287
Epoch 3/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.6167 - loss: 0.8679 - val_accuracy: 0.6092 - val_loss: 0.8840
Epoch 4/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.6158 - loss: 0.8605 - val_accuracy: 0.6113 - val_loss: 0.8717
Epoch 5/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.6337 - loss: 0.8409 - val_accuracy: 0.6273 - val_loss: 0.8672
Epoch 6/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.6475 - loss: 0.8253 - val_accuracy: 0.6166 - val_loss: 0.8712
Epoch 7/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.6343 - loss: 0.8187 - val_accuracy: 0.6177 - val_loss: 0.8589
Epoch 8/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.6456 - loss: 0.8211 - val_acc

In [108]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6488 - loss: 0.8383
Test Accuracy: 0.6294


In [110]:
import re

def predict_gender(name):
    name = name.lower()
    name = re.sub('[^a-z]', '', name)  # remove non-alphabet characters
    seq = tokenizer.texts_to_sequences([name])
    padded = pad_sequences(seq, maxlen=max_len, padding='post')
    pred = model.predict(padded)
    pred_class = pred.argmax(axis=1)[0]
    return le.inverse_transform([pred_class])[0]

# Example
print(predict_gender("Jessica"))  # ➜ Female
print(predict_gender("Aarav"))    # ➜ Male


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Female
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
Male
